Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Midpoint rule for computing integrals

Implement a routine to compute an integral 

$$
\int_a^b\! f(x)\,dx
$$

for a given $f(x)$ and integration limits $a$ and $b$, using the midpoint rule. Use a uniform mesh with $N$ elementary intervals.

In [13]:
def midpoint_rule(func, a, b, n):
    r"""Calculate the integral of `func` from `a` to `b` using the midpoint rule.
    
    Parameters
    ----------
    func : callable
        The function to integrate.
    a : float
        The lower limit of integration.
    b : float
        The upper limit of integration.
    n : int
        The number of intervals
        
    Returns
    -------
    integral : float
        The estimate of $\int_a^b f(x) dx$.
    """
    n = n + 1
    integral = 0

    for i in range(n):
        x_mid = (b - a) / n * (.5 + i)
        integral += func(x_mid)

    integral = integral * (b - a) / n

    return integral

In [14]:
# Test your routine

import numpy as np
from numpy.testing import assert_allclose

assert_allclose(midpoint_rule(lambda x: x ** 3, 0, 1, 100),
                1 / 4, atol=1e-3)
assert_allclose(midpoint_rule(lambda x: x ** 3, 0, 1, 1000),
                1 / 4, atol=1e-5)

Apply your routine to a simple function, where you can easily calculate the exact answer with paper and pencil. 
Study the error (i.e., the difference between the numerical estimate and an exact answer) at varying number of mesh points, $N$. What is the scaling of the error with $N$, is it $O(1/N)$ or $O(1/N^2)$ or something else? 

In [15]:
for n in [10, 100, 1000, 10000, 100000]:
    err = midpoint_rule(lambda x: x ** 3, 0, 1, n) - 0.25
    print("%6d -- %7.4g" % (n, err))

    10 -- -0.001033
   100 -- -1.225e-05
  1000 -- -1.248e-07
 10000 -- -1.25e-09
100000 -- -1.25e-11


## Integrable singularities

Now consider a more complicated intergral which has an integrable singularity at the lower limit of integration:


$$
I = \int_0^1\! \frac{\sin\sqrt{x}}{x}\,dx
$$


Applying your midpoint rule routine, we note that the convergence is rather poor.

In [16]:
from math import sin, sqrt


def func(x):
    return sin(sqrt(x)) / x

In [17]:
# Use a black-box library routine to compute the "ground truth" answer
from scipy.integrate import quad

q, err = quad(func, 0, 1)
print("the 'ground truth' value : %6f " % q, "\n")

the 'ground truth' value : 1.892166  



In [18]:
# Use the `midpoint_rule`

print("     N     error")
for n in [10, 100, 1000, 10000, 100000]:
    val = midpoint_rule(func, 0, 1, n)
    print("%6d -- %7.3g" % (n, val - q))

     N     error
    10 --  -0.182
   100 -- -0.0602
  1000 -- -0.0191
 10000 -- -0.00605
100000 -- -0.00191


To improve convergence, we can use the following trick: add and subtract the integral of some auxilliary function, so that the original integral $I$ splits into two integrals: one is regular and singularity-free, and the other one is easy to work out with paper and pencil. Use your `midpoint_rule` routine for the regular part.

In [24]:
def Int(x):
    return (np.sin(np.sqrt(x)) - np.sqrt(x)) / x

In [29]:
def compute_I(n):
    """Compute the integral $I$ using the midpoint rule. Subtract the singularity.
    
    Parameters
    ----------
    n : int
       The number of elementary intervals.
       
    Returns
    -------
    the estimate for the integral $I$.
    """

    I = 2
    I += midpoint_rule(Int, 0, 1, n)

    return I

In [30]:
q, err = quad(func, 0, 1)

# Test that your improved routine performs better than the naive application of the midpoint rule. 
#
# NOTE that your function should use your midpoint rule routine and
# it **must not** use library functions (`quad` et al).
#

for n in [10, 100, 1000]:
    naive = midpoint_rule(func, 0, 1, n)
    improved = compute_I(n)
    assert abs(improved - q) < abs(naive - q)

assert_allclose(compute_I(100), q, atol=1e-3)
assert_allclose(compute_I(1000), q, atol=1e-6)